###  Using the follwing to lunch gateway
https://interactivebrokers.github.io/cpwebapi/quickstart

1. cd clientportal.gw 
1. bin/run.sh root/conf.yaml
1. https://localhost:5000 
1. https://localhost:5000/demo#/
1. https://www.interactivebrokers.com/api/doc.html
1. https://localhost:5000/v1/api/iserver/contract/270639/info

### Objective:

Pull a csv file from IBKR containing portfilio data and creating 

1. porfolio performance, return
1. inflow, outflow

In [ ]:
# from jupyterthemes import get_themes
# import jupyterthemes as jt
# from jupyterthemes.stylefx import set_nb_theme
# # onedork
# # grade3
# # oceans16
# # chesterish
# # monokai
# # solarizedl
# # solarizedd
# set_nb_theme('onedork')

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_datareader
import os
import sys
import requests
import json
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import import_ipynb
from general_functions import *

In [ ]:
# Import the necessaries libraries
# import plotly.express as px
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

# Set notebook mode to work in offline
# import plotly.offline as pyo
# pyo.init_notebook_mode()

# from IPython.display import display, HTML
# pd.options.display.max_rows = 400
# pd.options.display.max_columns = 400
# pd.options.display.float_format = '{:,.2f}'.format
# display(HTML("<style>.container { width:96% !important; }</style>"))
# display(HTML("<style>div.output_scroll { height: 50em; }</style>"))

### Meaninful col
1. Allocation by Financial Instrument
1. Allocation by Asset Class
1. Deposits And Withdrawals
1. Performance by Asset Class

### Functions General  

In [ ]:
def post_jsonparsed_data(url,
                        json,
                        accountId=None):
    "post request via url"
    url = url.replace("http","https")
    if accountId != None:
        url = url.format(accountId=accountId)
    result = requests.post(url,
                           json= json,
                           verify=False).json()
    return(result)


def get_jsonparsed_data(url,
                        params=None,
                        accountId=None):
    "get request via url"
    url = url.replace("http","https")
    if accountId != None:
        url = url.format(accountId=accountId)
    if params != None:
        req_get_res = requests.get(url, params=params,verify=False).json()
    if params == None:
        req_get_res = requests.get(url, verify=False).json()
    return(req_get_res)

### Functions: CSV Specific 

In [ ]:
def ibkr_data_reader(path_of_data):
    df_data  = pd.read_csv(path_of_data,
                                     usecols=list(range(0,15)),
                                     names=list(range(0,15)))
    return(df_data)

In [ ]:
def ibkr_csv_data_parser(df_csv,
                         variable_name,
                         date_col_name = None,
                         column_type_dic=None):
    """Take the raw CSV data and parse the values"""
    df_data = df_csv.copy()
    df_allocation = df_data[(df_data[0] == variable_name)
                           ].reset_index(drop=True)[1:].dropna(axis=1)
    df_allocation = df_allocation.rename(columns = df_allocation.iloc[0]).drop(df_allocation.index[0]).reset_index(drop=True)
    if date_col_name != None:
        df_allocation[date_col_name] = pd.to_datetime(df_allocation[date_col_name])
    if column_type_dic != None:
        for col, data_type in zip(column_type_dic.keys(),column_type_dic.values()):
            df_allocation[col]  = df_allocation[col].astype(data_type)
    return(df_allocation)

### Functions with Rest API 

In [ ]:
def reauthenticate():
    return(requests.post('https://localhost:5000/v1/api/iserver/reauthenticate',verify=False).json())
    
def ping():
    return(requests.post('https://localhost:5000/v1/api/tickle',verify=False).json())

In [ ]:
def symbols_attribute_finder(symbols):
    """symbols: securities symnols all in one text seprated by comas such as 'INTC,XOM,AAPL'
    '"""
    ping()
    reauthenticate()
    ping()
    res  = get_jsonparsed_data("http://localhost:5000/v1/api/trsrv/stocks",
                               params={"symbols":[{symbols}]})
    
    set_of_symbols_passed = set(symbols.split(","))
    list_of_keys = list(res.keys())
    list_of_values = list(res.values())
    list_of_df_of_symbol_attributes = []
    for i in range(len(list_of_keys)):
        try:
            df_res = pd.DataFrame(list_of_values[i])
            df_res['symbol'] = list_of_keys[i]
            l = df_res['contracts'].values
            flat_list = [item for sublist in l for item in sublist]
            df_contract = pd.DataFrame(flat_list)
            df_res  = pd.concat([df_res,df_contract],axis=1).drop(["contracts"],axis=1)
            list_of_df_of_symbol_attributes.append(df_res)
        except:
            pass
    df = pd.concat(list_of_df_of_symbol_attributes)
    set_of_symbols_no_found =set_of_symbols_passed - set(df['symbol'])
    if len(set_of_symbols_no_found)>0:
        print("Did not find symbols",set_of_symbols_no_found)
    return(df)

def conid_atttribute_finder(list_of_conids):
    """Get attributes of Conids"""
    list_of_con_id_attribute_df =[]
    url = "https://localhost:5000/v1/api/iserver/contract/{conid}/info"
    for conid in list_of_conids:
        url_parameterized = url.format(conid=conid)
        res  = requests.get(url_parameterized,verify=False).json()
        df = pd.DataFrame([res])
        list_of_con_id_attribute_df.append(df)
    df_conid_att = pd.concat(list_of_con_id_attribute_df,ignore_index=True)
    return(df_conid_att)

In [ ]:
def pnl(accountId='123456789'):
    ping()
    reauthenticate()
    ping()
    res = get_jsonparsed_data(url="http://localhost:5000/v1/api/iserver/account/pnl/partitioned",
                              params={"acctId": {accountId}},
                              accountId=None)
    df_pnl = pd.DataFrame(res['upnl']).reset_index().rename(columns={"index":"variable"})
    return(df_pnl)


def portfolio_allocation(accountId='123456789'):
    ping()
    reauthenticate()
    ping()
    """Return dataframe breakdown of allocation within portoflio
    url:http://localhost:5000/v1/api/portfolio/{accountId}/allocation """
    ### high level portfolio allocation long/short in base currency
    res_allocation = get_jsonparsed_data("http://localhost:5000/v1/api/portfolio/{accountId}/allocation",
                                         accountId=accountId)
    df_allocation_asset_class = pd.DataFrame(res_allocation['assetClass']).sort_values("long",ascending=False).reset_index().rename(columns={"index":"assetClass"})
    df_allocation_sector = pd.DataFrame(res_allocation['sector']).sort_values("long",ascending=False).reset_index().rename(columns={"index":"sector"})
    df_allocation_group = pd.DataFrame(res_allocation['group']).sort_values("long",ascending=False).reset_index().rename(columns={"index":"group"})
    return({'df_allocation_asset_class':df_allocation_asset_class,
            'df_allocation_sector':df_allocation_sector,
            'df_allocation_group':df_allocation_group})


def account_performance(list_of_acctIds,
                        freq="D"):
    ping()
    reauthenticate()
    ping()
    """Return daily performance of the portfolio"""
    ### Portfoloio Performance
    res_performance = requests.post("https://localhost:5000/v1/api/pa/performance",
                                    json= {"acctIds": list_of_acctIds,
                                           "freq": freq},
                                    verify=False).json()
    navs = res_performance['nav']['data'][0]['navs']
    dates = res_performance['nav']['dates']
    returns = res_performance['cps']['data'][0]['returns']
    df_performance = pd.DataFrame({"dates":dates,
                                   "navs":navs,
                                   "returns":returns})
    df_performance['dates'] = pd.to_datetime(df_performance['dates'])
    return(df_performance)

In [ ]:
def my_current_holdings(accountId):
    ping()
    reauthenticate()
    ping()
    """Return dataframe of positions inm a portoflio 
    http://localhost:5000/v1/api/portfolio/{accountId}/positions/0
    ### metirc to add 
    ['market_capMM','pe_ratio','pb_ratio','dividend_yield','percentage','CumPercent']
    """
    ### this dict is to rename the columns of hoding ibkr same as columns name in Robinhood 
    dict_of_cols_to_convert = {"contractDesc":"symbol",
                               "group":"industry",
                               "baseMktPrice":"price",
                               "baseAvgCost":"average_buy_price",
                               "position":"quantity",
                               "baseMktValue":"equity",
                               "baseUnrealizedPnl":"equity_change"}
    ### The following must be ran first in order to get the position byd eatil with base currency
    portfolio_allocation(accountId = accountId)
    df_position_high_level = pd.DataFrame(get_jsonparsed_data("http://localhost:5000/v1/api/portfolio/{accountId}/positions",
                                                              accountId=accountId))
    ### portfolio position by detail
    ### you may need to write a for loop tp pull additional pages - it seems like each page gives 100 holding
    df_my_current_holding = pd.DataFrame(get_jsonparsed_data("http://localhost:5000/v1/api/portfolio/{accountId}/positions/0",
                                     accountId=accountId)).sort_values("baseMktValue",ascending=False).reset_index(drop=True)
    df_my_current_holding = df_my_current_holding.rename(columns=dict_of_cols_to_convert)
    ### stor list of conid to pass to conid_atttribute_finder
    list_of_conid = list(set(df_my_current_holding['conid']))
    print("Getting local symbols by conids")
    df_conid_att = conid_atttribute_finder(list_of_conid)
    df_my_current_holding = pd.merge(df_my_current_holding,
                                     df_conid_att[['con_id','local_symbol']].rename(columns={"con_id":"conid"}),
                                     left_on=['conid'],
                                     right_on=['conid'],
                                     how='left')
    ### portfolio calculation
    df_my_current_holding['equity_initial'] = df_my_current_holding['equity'] - df_my_current_holding['equity_change']
    df_my_current_holding['percent_change'] = ((df_my_current_holding['price']/df_my_current_holding['average_buy_price']) - 1)* 100
    df_my_current_holding['percentage'] =(df_my_current_holding['equity']/df_my_current_holding['equity'].sum())*100
    df_my_current_holding['CumPercent'] = df_my_current_holding['percentage'].cumsum()
    print("Getting holding attributes form yahoo finance")
    df_symbol_attributes_from_yahoo = symbol_attribute_retriever_from_yahoo(list(set(df_my_current_holding['local_symbol'])))
    df_my_current_holding = pd.merge(df_my_current_holding,
                                     df_symbol_attributes_from_yahoo.rename(columns={"symbol":'local_symbol'}),
                                     left_on=['local_symbol'],
                                     right_on=['local_symbol'],
                                     how='left',
                                    suffixes=("_ibkr","_yahoo"))
    print("retrieved attributes form yahoo finance",df_symbol_attributes_from_yahoo.shape)
    ### Creating columns of sector and industry from yahoo finance
    df_my_current_holding['name_ibkr'] = df_my_current_holding['name'].copy()
    df_my_current_holding['industry'] = df_my_current_holding['industry_yahoo'].fillna(df_my_current_holding['industry_ibkr'])
    df_my_current_holding['sector'] = df_my_current_holding['sector_yahoo'].fillna(df_my_current_holding['sector_ibkr'])
    df_my_current_holding['name'] = df_my_current_holding['shortName'].fillna(df_my_current_holding['name_ibkr'])
    ### reconcile Sector and Industry b etwene IBKR and ayhoo finance
    df_my_current_holding['sector'] = df_my_current_holding['sector'].replace({"Industrial":"Industrials"})
    
    cols_to_return=['symbol', 'name', 'sector', 'industry', 'returnOnAssets','returnOnEquity','quantity', 
                    'equity','equity_initial','equity_change','percent_change','percentage','CumPercent',
                    'average_buy_price','price','ev_ebit','firm_to_ebit','trailingPE','forwardPE','bookValue','priceToBook',
                    'ev','marketCap','netDebt','totalDebt','totalCash','totalRevenue','ebit','netIncomeToCommon','grossMargins','operatingMargins','profitMargins',
                    'currentRatio','quickRatio',
                    'grossProfits','ebitda','operatingCashflow','ebitdaMargins','freeCashflow',
                    'heldPercentInsiders','dividendYield','fiveYearAvgDividendYield','payoutRatio','symbolCurrency',
                    #'longBusinessSummary',
                    'fullTimeEmployees','debtToEquity','financialCurrency','exchange']
    df_my_current_holding = df_my_current_holding[cols_to_return].copy()
    df_my_current_holding.rename(columns={"returnOnAssets":"ROA",
                                            "returnOnEquity":"ROE",
                                            "average_buy_price":"ave_price",
                                             "bookValue":"pb",
                                             "priceToBook":"pb_ratio",
                                             "trailingPE":"pe",
                                             "forwardPE":'fpe',
                                            'totalRevenue':'rev',
                                            'operatingCashflow':'ocf',
                                            'dividendYield':'div_yeild',
                                          'netIncomeToCommon':'netIncome',
                                         'fiveYearAvgDividendYield':'5y_div_yeild'},inplace=True)
    return(df_my_current_holding)

### Functions with Applications 

In [ ]:
def portfolio_history(path_of_data = 'youfile_since_Inception_June_08_2023.csv'):
    ### NAV
    df_data  = ibkr_data_reader(path_of_data)
    print("Reading Portfolio Values (NAVS) from CSV")
    df_nav = ibkr_csv_data_parser(df_data,
                                         variable_name = 'Allocation by Financial Instrument',
                                         date_col_name='Date',
                                         column_type_dic = {'NAV':float,
                                                            'Cash':float,
                                                            'Stocks':float})
    print("Last NAV Value in CSV",df_nav['Date'].max())
    
    ### Getting Depoists
    ### Deposit
    print("Reading deposits")
    df_deposit = ibkr_csv_data_parser(df_data,
                             variable_name = 'Deposits And Withdrawals',
                                      date_col_name='Date',
                             column_type_dic = {'Amount':float})
    ## Net Deposit
    df_deposit_summary = df_deposit.groupby(['Date'])['Amount'].sum().reset_index(name='Net Deposit')

    
    df_portfolio_hist = pd.merge(df_nav.drop(['Header'],axis=1),
                                 df_deposit_summary,
                                 left_on='Date',
                                 right_on='Date',
                                 how='outer').fillna(0) 
    df_portfolio_hist['Cum Net Deposit'] = df_portfolio_hist['Net Deposit'].cumsum()
    df_portfolio_hist['CumPnL'] = df_portfolio_hist['NAV'] - df_portfolio_hist['Cum Net Deposit']
    df_portfolio_hist['PnL'] = df_portfolio_hist['CumPnL'].diff().fillna(0)
    df_portfolio_hist['adjusted_close_equity']= df_portfolio_hist['CumPnL'] + df_portfolio_hist['Net Deposit'].sum()
    df_portfolio_hist['Return'] = df_portfolio_hist['adjusted_close_equity'].pct_change().fillna(0)
    df_portfolio_hist['CumReturn']= ((df_portfolio_hist['Return'].fillna(0)+1).cumprod() - 1) * 100
    df_portfolio_hist = df_portfolio_hist.rename(columns={"Date":"begins_at","NAV":"close_market_value"})
    return(df_portfolio_hist)

In [ ]:
def my_portfolio(path_of_data,accountId):
    ### retrieve portfolio history
    df_portfolio_hist = portfolio_history(path_of_data =  path_of_data)
    df_my_current_holding = my_current_holdings(accountId)
    ### Create an empty datafrme for open orders  until I will figure out on how to implement it
    df_open_posotion = pd.DataFrame(columns=['symbol', 'name', 'side', 'type', 'current_price', 'price', 'quantity',
                                             'amount', 'quantity_current_holding',
                                             'average_price_current_holding', 'equity_current_holding', 'state','created_at'])
    print("portfolio info data retrieval compelted")
    return({"my_current_holdings":df_my_current_holding,
            "open_orders":df_open_posotion,
            "portfolio_history":df_portfolio_hist})

In [ ]:
def traded_contract(path_of_data):
    """Get all trades ever made | buy, sell"""
    df_trade_summary = ibkr_csv_data_parser(ibkr_data_reader(path_of_data = path_of_data),
                                            variable_name = 'Trade Summary')
    ### Japasnese stock syombols are ending iwth ".T" replacing the "".T" with None
    df_trade_summary['symbol_cleaned'] = df_trade_summary['Symbol'].str.replace(".T","")
    list_of_all_symbols_traded = list(set(df_trade_summary[df_trade_summary['Financial Instrument']!='Forex']['symbol_cleaned']))
    list_of_all_symbols_traded_as_text = ','.join(map(str,list_of_all_symbols_traded))
    print(list_of_all_symbols_traded_as_text)
    df_anything_traded = symbols_attribute_finder(list_of_all_symbols_traded_as_text)
    return (df_anything_traded)

In [ ]:
def tansaction_history_provider(accountId,
                                conid,
                                days=7000):
    df_trans_hist_conid = pd.DataFrame(post_jsonparsed_data(url="http://localhost:5000/v1/api/pa/transactions",
                                      json={"acctIds": [accountId],
                                            "conids": [conid],
                                            "currency": "USD",
                                            "days": days})['transactions'])
    return(df_trans_hist_conid)

In [ ]:
def string_to_datetime(s):
    try:
        s = dt.datetime.strptime(s,'%a %b %d %X %Z %Y')
    except:
        pass
    try:
        s = pd.to_datetime(s,format='mixed')
    except:
        pass
    return(s)

In [ ]:
def trade_histroy(path_of_data,
                  accountId,
                  days=7000):
    """provide histroial trades by date, any isntrument ever traded"""
    df_anything_traded = traded_contract(path_of_data)
    list_conid = list(set(df_anything_traded['conid']))
    
    list_of_anything_traded_conid = list(set(df_anything_traded['conid']))
    list_of_df_transaction_history = []
    print("Getting trade history")
    for con in list_of_anything_traded_conid:
        #print(con)
        try:
            df_tran = pd.DataFrame()
            df_tran = tansaction_history_provider(accountId = accountId,
                                             conid=con)
        except:
            pass
        list_of_df_transaction_history.append(df_tran)
    df = pd.concat(list_of_df_transaction_history,
                   ignore_index=True)
    print("Trade history retrieval completed")
    df_trade_data = pd.merge(df,
                             df_anything_traded[['conid','symbol','name','assetClass','exchange']],
                             left_on=['conid'],
                             right_on=['conid'],
                             how='left')
    df_trade_data['date'] = df_trade_data['date'].apply(lambda s: string_to_datetime(s))
    df_trade_data['trade_amount'] = df_trade_data['amt']*df_trade_data['fxRate']
    ### Create a trade summary pivot table
    print("Summerizing Trade")
    df_trade_summary = pd.pivot_table(df_trade_data,
                                    index='date',
                                   values='trade_amount',
                                   columns=['type'],
                                   aggfunc='sum').reset_index().fillna(0)
    df_trade_summary['trade_amount buy'] = (df_trade_summary['Buy'].fillna(0) + df_trade_summary['Buy to Cover'].fillna(0))*-1
    df_trade_summary['trade_amount sell'] = (df_trade_summary['Sell'].fillna(0) + df_trade_summary['Sell Short'].fillna(0))*-1
    df_trade_summary['Net'] = df_trade_summary['trade_amount buy'] + df_trade_summary['trade_amount sell']
    print("Summerizing Trade completed")
    return({"df_trade_data":df_trade_data,
            "df_trade_summary":df_trade_summary})

###  How to use it

In [ ]:
# my_portfolio_dict = my_portfolio(path_of_data= csv_file_name, accountId=account_number)

In [ ]:
# trade_histroy_dict = trade_histroy(path_of_data = csv_file_name,
#                                    accountId = account_number,
#                                    days=7000)

In [ ]:
# portfolio_analytics(my_portfolio_dict,trade_histroy_dict)